In [ ]:
alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
# alph = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
# alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя"

len_alph = len(alph)

In [45]:
class Playfair:
    def __init__(self, breakdown: str = 'Х', matrix_size: str = (4, 8), visual: bool = True):
        self.breakdown = breakdown
        self.matrix: list[str] = list()
        self.matrix_size = matrix_size
        self.visual = visual

    def set_matrix(self, phrase: str) -> None:
        lines_for_phrase = len(phrase) // self.matrix_size[0]
        ost_phrase = phrase[len(phrase) - 1 - (len(phrase) % self.matrix_size[0]):]
        for i in range(lines_for_phrase):
            self.matrix.append(phrase[i*self.matrix_size[0]:(i+1)*self.matrix_size[0]])
        phrase_set = set(phrase)
        ost_alph = "".join([i for i in alph if i not in phrase_set])
        ost = ost_phrase + ost_alph
        ost_lines = self.matrix_size[1] - lines_for_phrase
        for i in range(ost_lines):
            self.matrix.append(ost[i*self.matrix_size[0]:(i+1)*self.matrix_size[0]])
        
    def print_matrix(self) -> None:
        for line in self.matrix:
            print(*list(line), sep=" ")

    def prepare_data_to_encrypt(self, data: str) -> list[str]:
        result = []
        last = None
        for i in data:
            if last is None:
                last = i
                continue

            if last == i:
                result.append(last + self.breakdown)
                last = self.breakdown
                continue
                
            result.append(last + i)
            last = None
        
        if last is not None:
            result.append(last + self.breakdown)
        
        return result
    
    def _encrypt_bigram(self, bi: str) -> str:
        bi1 = [None, None]
        bi2 = [None, None]

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[0])) != -1:
                bi1 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[1])) != -1:
                bi2 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        if bi1[0] == bi2[0]:
            bi1[1] = (bi1[1] + 1) % self.matrix_size[0]
            bi2[1] = (bi2[1] + 1) % self.matrix_size[0]
        elif bi1[1] == bi2[1]:
            bi1[0] = (bi1[0] + 1) % self.matrix_size[1]
            bi2[0] = (bi2[0] + 1) % self.matrix_size[1]
        else:
            bi1, bi2 = (bi2[0], bi1[1]), (bi1[0], bi2[1])
        return self.matrix[bi1[0]][bi1[1]] + self.matrix[bi2[0]][bi2[1]]
    
    def encrypt(self, data: str) -> str:
        prepared_data = self.prepare_data_to_encrypt(data)
        encrypted_data = ""
        for bigram in prepared_data:
            encrypt_bigram = self._encrypt_bigram(bigram)
            if self.visual:
                print(bigram, "->", encrypt_bigram)
            encrypted_data += encrypt_bigram
        return encrypted_data



In [46]:
cypher = Playfair()
cypher.set_matrix("ПРИВЕТ")
cypher.print_matrix()

П Р И В
В Е Т А
Б Г Д Ё
Ж З Й К
Л М Н О
С У Ф Х
Ц Ч Ш Щ
Ъ Ы Ь Э


In [ ]:
cypher.encrypt("ЧТОНИБУДЬ")

ДА -> ТЁ
ЙМ -> НЗ
АН -> ОТ
КИ -> ВЙ


'ТЁНЗОТВЙ'